### 搭建倒排表
倒排表的作用是让搜索更加快速，是搜索引擎中常用的技术。根据课程中所讲的方法，你需要完成这部分的代码。 

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
from gensim.models import KeyedVectors  # 词向量用来比较俩俩之间相似度

In [2]:
# 读取数据： 导入在preprocessor.ipynb中生成的data/question_answer_pares.pkl文件，并将其保存在变量QApares中
with open('data/question_answer_pares.pkl','rb') as f:
    QApares = pickle.load(f)

In [3]:
QApares.head()

,question,answer,question_after_preprocessing
0,买二份有没有少点呀,亲亲真的不好意思我们已经是优惠价了呢小本生意请亲谅解,"[买, 二份, 有没有, 少点]"
1,那就等你们处理喽,好的亲退了,[处理]
2,那我不喜欢,颜色的话一般茶刀茶针和二合一的话都是红木檀和黑木檀哦,[喜欢]
3,不是免运费,本店茶具订单满99包邮除宁夏青海内蒙古海南新疆西藏满39包邮,"[免, 运费]"
4,好吃吗,好吃的,[好吃]


```TODO1``` 构造一个倒排表，不需要考虑单词的相似度

In [4]:
# 构建一个倒排表，有关倒排表的详细内容参考实验手册
# 为了能够快速检索，倒排表应用哈希表来存储。python中字典内部便是用哈希表来存储的，所以这里我们直接将倒排表保存在字典中
# 注意：在这里不需要考虑单词之间的相似度。
inverted_list = {}
for index,sentence in enumerate(QApares.question_after_preprocessing):
    ### 你需要完成的代码
    if type(sentence)==list:
        for word in sentence:
            if word not in inverted_list:
                inverted_list[word] = [index]
            else:
                inverted_list[word].append(index)
    
    ### 你需要完成的代码结束

In [5]:
#d ata/retrieve/sgns.zhihu.word是从https://github.com/Embedding/Chinese-Word-Vectors下载到的预训练好的中文词向量文件
#使 用KeyedVectors.load_word2vec_format()函数加载预训练好的词向量文件
model = KeyedVectors.load_word2vec_format('data/retrieve/sgns.zhihu.word')

In [6]:
def get_similar_by_word(word,topk):
    '''
        返回与一个单词word相似度最高的topk个单词所组成的单词列表
        出参：
            word_list：与word相似度最高的topk个单词所组成的单词列表。格式为[单词1，单词2，单词3，单词4，单词5]
    '''
    similar_words = model.similar_by_word(word,topk)
    word_list = [word[0] for word in similar_words]
    return word_list

```TODO2``` 构造一个新的倒排表，考虑单词之间的语义相似度

In [7]:
# TODO：
# 构造一个新的倒排表，并将结果保存在字典inverted_list_new中
# 新的倒排表键为word，值为老倒排表[word]、老倒排表[单词1]、老倒排表[单词2]、老倒排表[单词3]、老倒排表[单词4]的并集
# 即新倒排表保存了包含单词word或包含与单词word最相近的5个单词中的某一个的问题的index
inverted_list_new = {}
for word in tqdm(inverted_list):
    ### 你需要完成的部分
    inverted_list_new[word] = set(inverted_list[word])
    if word in model.vocab:
        similar_words = get_similar_by_word(word,5)
        for similar_word in similar_words:
                if similar_word in inverted_list:
                    inverted_list_new[word] = inverted_list_new[word] | set(inverted_list[similar_word])
    ### 你需要完成的代码结束
    

100%|██████████| 3832/3832 [00:58<00:00, 65.82it/s]


In [8]:
# 将新的倒排表保存在文件data/retrieve/invertedList.pkl中
with open('data/retrieve/invertedList.pkl','wb') as f:
    pickle.dump(inverted_list_new,f)

以下为测试，完成上述过程之后，可以运行以下的代码来测试准确性。

In [9]:
#这一格的内容是从preprocessor.ipynb中粘贴而来，包含了数据预处理的几个关键函数
import emoji
import re
import jieba
def clean(content):
    content = emoji.demojize(content)
    content = re.sub('<.*>','',content)
    return content
#这一函数是用于对句子进行分词，在preprocessor.ipynb中由于数据是已经分好词的，所以我们并没有进行这一步骤，但是对于一个新的问句，这一步是必不可少的
def question_cut(content):
    return list(jieba.cut(content))
def strip(wordList):
    return [word.strip() for word in wordList if word.strip()!='']
with open("data/stopWord.json","r") as f:
    stopWords = f.read().split("\n")
def rm_stop_word(wordList):
    return [word for word in wordList if word not in stopWords]

In [10]:
# 从data/retrieve/invertedList.pkl加载倒排表并将其保存在变量invertedList中
with open('data/retrieve/invertedList.pkl','rb') as f:
    invertedList = pickle.load(f)

In [11]:
def get_retrieve_result(sentence):
    '''
        输入一个句子sentence，根据倒排表进行快速检索，返回与该句子较相近的一些候选问题的index
        候选问题由包含该句子中任一单词或包含与该句子中任一单词意思相近的单词的问题索引组成
    '''
    sentence = clean(sentence)
    sentence = question_cut(sentence)
    sentence = strip(sentence)
    sentence = rm_stop_word(sentence)
    candidate = set()
    for word in sentence:
        if word in invertedList:
            candidate = candidate | invertedList[word]
    return candidate

In [12]:
get_retrieve_result('什么时候发货')  # 通过倒排表返回文档IDs

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/qq/1bp7zzf91vbd_tlf36jvpdbm0000gn/T/jieba.cache
Loading model cost 0.675 seconds.
Prefix dict has been built successfully.


{81920,
 16386,
 65541,
 5,
 81927,
 32776,
 81930,
 81935,
 17,
 18,
 65554,
 16401,
 98331,
 81947,
 29,
 65566,
 32800,
 81953,
 32803,
 98339,
 81959,
 32810,
 98346,
 49194,
 32818,
 16435,
 55,
 49209,
 98366,
 64,
 49219,
 65604,
 81988,
 16458,
 65611,
 81995,
 81998,
 16463,
 16464,
 49233,
 32850,
 98387,
 98386,
 49234,
 86,
 81999,
 82004,
 32859,
 16475,
 49245,
 98398,
 65631,
 65630,
 82015,
 102,
 65639,
 65640,
 49259,
 65646,
 98415,
 98416,
 49263,
 65650,
 16495,
 49267,
 16500,
 118,
 82035,
 65656,
 16503,
 122,
 65659,
 65660,
 125,
 32894,
 49275,
 133,
 65669,
 65670,
 65671,
 32902,
 139,
 49293,
 142,
 65679,
 32912,
 98451,
 150,
 151,
 32929,
 49318,
 49320,
 65708,
 82092,
 82093,
 65711,
 98484,
 98489,
 49337,
 187,
 49340,
 32957,
 200,
 16588,
 32973,
 65742,
 16589,
 98518,
 16598,
 49366,
 82137,
 65755,
 220,
 223,
 65764,
 82149,
 82155,
 16621,
 49396,
 65783,
 33017,
 65786,
 254,
 65790,
 82176,
 261,
 65798,
 65806,
 65810,
 275,
 279,
 98586,
